# What is the meaning of ADA?

In [3]:
import random

answer = None
number_of_tries = 0
while answer != 42:
    # Generate one random number
    answer = random.randint(0, 1_000_000)
    number_of_tries += 1

print("Number of TEEEEEENTATIVES:", number_of_tries, end="\r")
print("\nThe answer to ADA, the universe and everything is:", answer)


Number of TEEEEEENTATIVES: 305944
The answer to ADA, the universe and everything is: 42


# <center> “Punchlines as Mirrors: Social Attitudes, Politics, and Biases in *The New Yorker* Caption Contest”

<center><b> Humor reflecting society’s views, stereotypes, and political climate</b></center>


## Narrative Flow

- **Introduction:** The Caption Contest as a cultural mirror — humor as social data.  
- **Axis 1:** Professions & politics → humor about authority and power, *“What are people laughing about?”*  
- **Axis 2:** Humor in time → historical & contextual dimensions, *“When and why do jokes resonate?”*  
- **Axis 3:** Social norms → gender roles & taboos, testing the limits of humor, *“What’s acceptable or not?”*  
- **Axis 4:** Biases → explain psychological and cultural mechanisms behind why we laugh, *“Why do we find it funny?”*  
- **Conclusion:** Humor not only entertains — it reveals evolving attitudes, biases, and the cultural pulse of society.

> **Idée pour le website:** chaque section devrait commencer par un ensemble de cartoons du concours pour plonger le spectateur dans l’humour avant de passer à l’analyse.

---
---

## 1. Professions, Politics, and Power

**Professions in Humor:**  
Which jobs are depicted most often? Which are ridiculed vs. admired? What stereotypes are recurrent (e.g., lawyers as tricksters, doctors as saviors)?  

**Politics in Humor:**  
Do captions reflect partisan leanings (Democrat vs. Republican) or mock political figures more broadly? Are political jokes rated differently?  

**Interplay:**  
Professions like politicians or lawyers sit at the crossroads of both — this axis highlights how authority and social roles are viewed through humor.

### Plots / Stats

- **Bar / Word Clouds:** Frequency of professions mentioned in captions (“doctor,” “lawyer,” “politician”).  
- **Histograms / Line Plots:** Frequency of professions across time.  
- **Grouped Bar Charts:** Average “funniness” scores by profession category (healthcare, law, politics, education, etc.).  
- **Heatmaps:** Cross-tab professions × sentiment (positive / negative / neutral).  
- **Cartoon + Caption Samples:** A few annotated cartoons showing how professions are ridiculed (adds storytelling color).  

**For politics:**
- Timeline of mentions of political figures / parties.  
- Sentiment distribution around Democrats vs. Republicans.  
- Example “political joke clusters” side by side with major events (e.g., elections).  

**Statistical Tests / Methods:**
- *t-tests / z-tests* → Compare funniness scores of politicians vs. other professions.  
- *Multiple hypothesis testing (FDR/BH)* → Control for comparisons across 30+ job categories.  
- *Network graphs* → Co-occurrence of profession keywords with stereotypes (“lawyer–money,” “doctor–death”).  
- *Linear regression / lmplot* → Test if political humor ratings rise around elections.  
- *Pearsonr / Spearmanr* → Correlation between real-world political cycles and joke frequency.  

---

## Preliminary Research

---
---

## 2. Humor in Context: Time, News, and Global Events

**Temporal Shifts:**  
How do joke styles evolve over decades of contests?  

**Event-Driven Humor:**  
Are certain joke clusters (e.g., pandemic jokes, economic crisis humor) tied to real-world events?  

**Editorial & Cultural Climate:**  
To what extent might editorial influence what becomes “funny” at a given moment?

### Plots / Stats

- **Timeline Charts:** Average funniness scores across years (detecting peaks/troughs).  
- **Cluster Maps (t-SNE / UMAP):** Humor types (e.g., puns, absurdity, political jokes) clustered and color-coded by time period.  
  - *(PCA / scatter plot matrix → map high-dimensional humor features into 2D time-evolving plots.)*  
- **Overlay with Historical Events:** Vertical lines marking pandemics, recessions, elections, etc.  
- **Rolling Average Trends:** How often humor about specific topics (health, money, environment) surfaces year by year.  
- **Line Plots / Stacked Plots:** Share of humor categories over decades.  
- **Clustering (k-means, hierarchical):** Humor style clusters from text embeddings.  
- **Jointplot:** Caption length vs. funniness, with year as hue.  
- **kstest:** Test if distributions of funniness scores shift significantly during crises vs. stable periods.  
- **Causality tests (Granger):** Do global events precede spikes in political/taboo humor?  
- **Regression with time covariates:** Predict funniness as function of time + event markers.  

---

## Preliminary Research

---
---

## 3. Social Representations and Boundaries of Humor

**Gender and Representation:**  
Who appears in the cartoons? How are men vs. women depicted (professional vs. domestic roles)?  
Do captions reinforce stereotypes, and does the audience reward or punish them?  

**Taboo Themes:**  
How often do jokes about religion, sex, death, or violence appear? Are they rated as funnier or riskier depending on the era?  

**Audience Sensitivity:**  
Do taboos lose their edge over time, or does shock humor stay consistently popular?

### Plots / Stats

- **Gender Representation Bars:** Share of male vs. female figures in images.  
- **Role Distribution Sankey Diagram:** Flow from gender → depicted roles (domestic, professional, heroic, villainous).  
- **Caption Analysis Word Clouds:** Gendered terms (e.g., “wife,” “husband,” “boss,” “nurse”) and their co-occurrences.  
- **Taboo Radar Chart:** Relative frequency of taboo themes (death, religion, sex, violence, race) across decades.  
- **Boxplots:** Funniness scores of taboo vs. non-taboo captions.  
- **Annotated Timeline:** Spikes in taboo humor aligned with real-world crises or cultural shifts.  

**Statistical Tests:**
- *t-tests / binomial-tests* → Gender differences in roles; taboo vs. non-taboo funniness.  
- *Spearman correlation* → Taboo frequency vs. political/economic crises.  
- *CCDF* → Heavy-tailed popularity of taboo jokes (shock jokes succeed rarely but spectacularly).  

---


## Preliminary Research

---
---

## 4. Cognitive and Cultural Biases in Humor

**Attentional Biases:**  
Do certain visuals (talking animals, surreal objects) attract more humorous captions? Do raters become “bored” after reading too many captions?  

**Judgment Biases:**  
Are logical vs. emotional jokes received differently? Do certain rhetorical structures (puns, absurdities) align with higher scores?  

**Cultural Biases:**  
Do references to celebrities, brands, or memes outperform timeless jokes? How do culturally specific jokes age across time?

### Plots / Stats

- **Voting Distribution Curves:** Detect the “bored-rater effect” (scores declining as voters progress through captions).  
- **Scatterplots / regplots:** Visual oddity (unusual objects, surreal images) vs. funniness scores.  
- **Polarity / Subjectivity Histograms:** Comparing logical vs. emotional jokes.  
- **Cultural Reference Heatmap:** Proper nouns (brands, celebrities, memes) frequency and funniness.  
- **Side-by-Side Caption Examples:** One “timeless” vs. one “culture-specific” joke, annotated to show differences.  
- **Spearmanr / Pearsonr:** Test correlation between rating order and score.  
- **Histograms / PDF, CDF:** Distribution of culturally referential vs. non-referential captions.  
- **Log / semilog plots:** Detect heavy-tailed patterns (a few jokes get very high scores).  
- **Logistic regression:** Predict caption success from presence of cultural references.  
- **Unsupervised clustering:** Group captions by bias-driven humor style.  
- **Network graph:** Links between cultural references (brands / celebrities) and funniness.

---

## Preliminary Research